In [29]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import itertools
from itertools import product



options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

In [30]:
K = 3
lambda_star = np.array([-1,3,1])

I = 20
J = 40
sigma = 1

### generate exogenous data

# np.random.seed(2)
# φ_i_j_k = np.random.normal(0,1, size=[I,J,K])
# eps_ij = sigma * np.random.normal(0,1, size = [I,J])
# def φ_k(i_idx,bundles):
#     return (φ_i_j_k[i_idx[:,None], bundles,:]).sum(1)



np.random.seed(4)
φ_i_j_k = np.random.normal(0,5, size=[I,J,K-1])
eps_i_j = sigma * np.random.normal(0,1, size = [I,J])

phi_j = np.random.normal(0,1, size=[J])**2
t_j_j = np.random.normal(0,1, size=[J,J])**2
t_j_sum = t_j_j.sum(1)
def φ_k(i_idx,bundles):
    complementarities_i_k = (phi_j[bundles] * (t_j_j[bundles[:,:,None] , bundles[:,None,:]]).sum(-1) 
                             / t_j_sum[bundles]).sum(-1)
    
    return np.hstack(((φ_i_j_k[i_idx[:,None], bundles,:]).sum(1), complementarities_i_k.reshape(-1, 1) ))

In [31]:
bundle = np.random.choice(np.arange(J), size=[2,5], replace=False)
print(bundle)
# φ_k(np.array([5,7]),bundle)

[[ 1 26  0  3 22]
 [36  8 27 11 14]]


In [32]:
def grad_lovatz_extension(i_idx, z_i_j, lambda_k, eps_i_j = None) :
    if eps_i_j is None:
        eps_i_j = np.zeros([len(i_idx),J])              

    sorted_z_id_j = z_i_j.argsort(1)[:,::-1]

    grad = np.zeros([len(z_i_j),J])
    for j in range(J):
        grad[np.arange(len(i_idx)),sorted_z_id_j[:,j]] = (φ_k(i_idx, sorted_z_id_j[:,np.arange(j+1)] ) @ lambda_k 
                                    - φ_k(i_idx, sorted_z_id_j[:,np.arange(j)] ) @ lambda_k  
                                    + eps_i_j[np.arange(len(i_idx)),sorted_z_id_j[:,j]])
    
    return grad


def maximize_supermodular(i_idx, lambda_k, num_iterations,alpha ,eps_i_j = None):

    z_t = np.ones([len(i_idx),J])/2 

    z_list = []
    iter = 0 
    for _ in range(num_iterations):
        grad = grad_lovatz_extension(i_idx, z_t, lambda_k, eps_i_j) 
        z_new = z_t + alpha * grad / np.linalg.norm(grad)
        z_new = np.clip(z_new, 0, 1)

        z_t = z_new
        z_list.append(z_new)
        iter += 1

    z_star = np.array(z_list)[-int(np.floor(num_iterations)/10):,:,:].mean(0)
    bundle_star = np.array(z_star.round(0), dtype= bool)

    return bundle_star, z_star


In [33]:
for i in range(I):
    bundle_star ,  z_star = maximize_supermodular(np.array([i]),lambda_star,600,1)

# print(z_star)

    print(z_star[(z_star > .1) & (z_star < .9)])    
print(bundle_star)
print(φ_k(np.array([0]),np.where(bundle_star)[0].reshape(1,-1)) @ lambda_star)

[]
[]
[]
[]
[]
[]
[0.88678817]
[]
[]
[]
[0.40744177 0.11145062]
[0.71144088]
[]
[]
[]
[]
[]
[]
[]
[]
[[False  True  True False  True False False False False  True False False
   True False  True False  True False False False  True  True  True  True
  False  True  True False False False False  True  True  True  True  True
   True  True False  True]]
[152.45172622]


In [34]:
def maximize_brute_force(i, lambda_k):
    max_value = -np.inf
    best_array = None

    # Generate all binary arrays of size J
    for binary_array in product([0, 1], repeat=J):
        B = np.where(np.array(binary_array, dtype=bool))[0]. reshape(1,-1)

        current_value = φ_k(np.array([i]),B) @ lambda_k

        # Check if the current value is less than the max_value
        if current_value > max_value:
            max_value = current_value
            best_array = B

    return max_value, best_array

# Find the maximizer and the maximum value
if J <= 15:
    max_value, maximizer = maximize_brute_force(0, lambda_star)

    print("maximum Value:", max_value)
    print("maximizer:", maximizer)

# Estimation

### Generate data

In [35]:
bundles_observed ,  z_star = maximize_supermodular(np.arange(I),lambda_star,500,1, eps_i_j)
print(z_star[(z_star > .1) & (z_star < .9)])  

phi_k_hat = 0
for i in range(I):
    phi_k_hat += φ_k(np.array([i]),np.where(bundles_observed[i])[0].reshape(1,-1))[0]

print(phi_k_hat )

[0.62731652 0.50514886 0.81069331 0.50536311 0.59463022 0.31087167
 0.44158134 0.72316802 0.72812371 0.57120442]
[-414.32504322 1529.97626092  198.54879962]


In [37]:
max_iters = 40
tol = 1e-12

np.random.seed(443)
S = 30
eps_si_j = sigma * np.random.normal(0,1, size = [S*I,J])      

with gp.Env(params=options) as env:
    with gp.Model(env=env) as model:
        ### Initialize 
        # Create variables
        lambda_k = model.addVars(K, lb= -GRB.INFINITY, ub = GRB.INFINITY , name="parameters")
        u_si = model.addVars(S*I, name="utilities")

        # Set objective
        model.setObjective(gp.quicksum(phi_k_hat[k] * lambda_k[k] for k in range(K))
                            - (1/S)* u_si.sum(), GRB.MAXIMIZE)
        
        model.addConstrs((u_si[si] >= 
                            gp.quicksum(φ_k(np.array([si // S]), np.where(bundles_observed[si // S])[0].reshape(1,-1))[0][k] *  lambda_k[k] for k in range(K))
                            + eps_si_j[si, bundles_observed[si // S]].sum()
                            for si in range(S*I)), name="constraint_batch")
        
        # Optimize the model
        model.setParam('OutputFlag', 0)
        # model.setParam('Presolve', 0)
        model.optimize()
        status = model.Status
        # Extract the solution 
        theta_solution = np.array(model.x)

        ### Column Generation
        iter = 0
        while iter < max_iters:
            print(f"ITER: {iter}, parameters: {theta_solution[:K]}")
            ### Pricing problem
            B_star_si = []
            val_si = []
            
            for s in range(S):
                        B_star, _ = maximize_supermodular(np.arange(I), theta_solution[:K], 500, 1, eps_si_j[s*I:(s+1)*I]) 
                        B_star_si.append(B_star)
            
            B_star_si = np.array(B_star_si).reshape([S*I,J])

            for si in range(S*I):
                  val_si.append(φ_k(np.array([si // S]), np.where(B_star_si[si])[0].reshape(1,-1)) @ theta_solution[:K] 
                                + eps_si_j[si, B_star_si[si]].sum() - theta_solution[K+si] )
            val_si = np.array(val_si)

            print(f"pricing done")

            # stop if certificate holds
            print(f"reduced cost: {np.max(val_si)}")
            if np.max(val_si) <= tol:
                primal_solutin = np.array(model.x)
                dual_solution = np.array(model.pi)
                break
            
            ### Master problem
            # add constraints
            model.addConstrs((u_si[si] >= 
                            gp.quicksum(φ_k(np.array([si // S]), np.where(B_star_si[si ])[0].reshape(1,-1))[0][k] *  lambda_k[k] for k in range(K))
                            + eps_si_j[si, B_star_si[si ]].sum()
                            for si in range(S*I)), name=f"constraint_batch_{iter}")
            
            # optimize the model
            lambda_k.start = theta_solution[:K]
            u_si.start = val_si + theta_solution[K:S*I +K]
            model.optimize()
            theta_solution = np.array(model.x)
            iter += 1
            print("##############")
            

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
ITER: 0, parameters: [-0.10687921  0.10297957  0.30328282]
pricing done
reduced cost: 16.50762957305961
##############
ITER: 1, parameters: [-0.01105103  0.62252431  0.47717267]
pricing done
reduced cost: 13.642558920582971
##############
ITER: 2, parameters: [-0.47532643  0.76026967 -0.1825308 ]
pricing done
reduced cost: 10.62444758704077
##############
ITER: 3, parameters: [-0.5311098   2.07126657  1.04511348]
pricing done
reduced cost: 7.559181153731117
##############
ITER: 4, parameters: [-1.72312567  5.15865247  1.80355987]
pricing done
reduced cost: 4.560985996689851
##############
ITER: 5, parameters: [-0.64952403  1.85866786  0.33002738]
pricing done
reduced cost: 1.3649209373719202
##############
ITER: 6, parameters: [-0.94329795  2.88668842  1.0050684 ]
pricing done
reduced cost: 0.6869027560688039
###

In [27]:
lambda_star

array([-1,  3,  1])

In [28]:
theta_solution[:K]

array([-1.28414983,  4.12236884,  1.35991002])